In [143]:
%cd "C:\Users\hrish\George Mason University - O365 Production\DAEN 690 - Group Only-GRP - 063022"

C:\Users\hrish\George Mason University - O365 Production\DAEN 690 - Group Only-GRP - 063022


In [144]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence

In [148]:
first_file = pd.read_csv('SimData_2022.06.30_10.27.04.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

second_file = pd.read_csv('SimData_2022.06.30_13.01.46.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

third_file = pd.read_csv('SimData_2022.06.30_13.05.32.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

fourth_file = pd.read_csv('SimData_2022.06.30_13.42.04.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

df = pd.concat([first_file, second_file, third_file, fourth_file])

In [149]:
df.head(10)
df['turnDirection'] = df['trueHeading'].rolling(window = 5).apply(lambda x: x[-1]-x[0], raw=True)
df['Altdiff'] = df['absoluteAltitude'].rolling(window = 5).apply(lambda x: x[0]- x[-1], raw = True)
df['groundSpeedDiff'] = df['groundSpeed'].rolling(window = 5).apply(lambda x: x[0]- x[-1], raw = True)
df['turnDirection']  = df['turnDirection'].fillna(0)
df['Altdiff'] = df['Altdiff'].fillna(0)
df['groundSpeedDiff']=df['groundSpeedDiff'].fillna(0)
df.head(10)


,datetime,_id,latitude,longitude,gpsAltitude,absoluteAltitude,magneticHeading,trueHeading,pitch,roll,...,hasTransmissionChipCaution,hasTransmissionOilTemperatureWarning,hasTransmissionOilPressureWarning,timestamp,gpsDateTime,seconds,vil,turnDirection,Altdiff,groundSpeedDiff
0,2022-06-30 14:27:53.216,::1656613673.216,20.904799,-156.424679,36.82,0.12,204.071,213.554,1.594,0.665,...,False,False,False,2022-06-30T14:27:53.216Z,2022-06-30T14:27:53.216Z,1.656614e+09,-2,0.000,0.0,0.000000
1,2022-06-30 14:27:53.234,::1656613673.234,20.904799,-156.424679,36.82,0.12,204.064,213.547,1.593,0.683,...,False,False,False,2022-06-30T14:27:53.234Z,2022-06-30T14:27:53.234Z,1.656614e+09,-2,0.000,0.0,0.000000
2,2022-06-30 14:27:53.251,::1656613673.251,20.904799,-156.424679,36.82,0.12,204.058,213.541,1.593,0.707,...,False,False,False,2022-06-30T14:27:53.251Z,2022-06-30T14:27:53.251Z,1.656614e+09,-2,0.000,0.0,0.000000
3,2022-06-30 14:27:53.271,::1656613673.271,20.904799,-156.424679,36.81,0.12,204.051,213.534,1.594,0.735,...,False,False,False,2022-06-30T14:27:53.271Z,2022-06-30T14:27:53.271Z,1.656614e+09,-2,0.000,0.0,0.000000
4,2022-06-30 14:27:53.293,::1656613673.293,20.904799,-156.424679,36.81,0.12,204.043,213.526,1.596,0.770,...,False,False,False,2022-06-30T14:27:53.293Z,2022-06-30T14:27:53.293Z,1.656614e+09,-2,-0.028,0.0,-0.030217
5,2022-06-30 14:27:53.313,::1656613673.313,20.904799,-156.424679,36.81,0.12,204.035,213.518,1.600,0.807,...,False,False,False,2022-06-30T14:27:53.313Z,2022-06-30T14:27:53.313Z,1.656614e+09,-2,-0.029,0.0,-0.036142
6,2022-06-30 14:27:53.334,::1656613673.334,20.904799,-156.424679,36.81,0.12,204.029,213.512,1.605,0.838,...,False,False,False,2022-06-30T14:27:53.334Z,2022-06-30T14:27:53.334Z,1.656614e+09,-2,-0.029,0.0,-0.030809
7,2022-06-30 14:27:53.356,::1656613673.356,20.904799,-156.424679,36.82,0.12,204.023,213.506,1.612,0.868,...,False,False,False,2022-06-30T14:27:53.356Z,2022-06-30T14:27:53.356Z,1.656614e+09,-2,-0.028,0.0,-0.020737
8,2022-06-30 14:27:53.376,::1656613673.376,20.904799,-156.424679,36.82,0.12,204.017,213.499,1.622,0.893,...,False,False,False,2022-06-30T14:27:53.376Z,2022-06-30T14:27:53.376Z,1.656614e+09,-2,-0.027,0.0,-0.007110
9,2022-06-30 14:27:53.393,::1656613673.393,20.904799,-156.424679,36.82,0.12,204.012,213.495,1.633,0.909,...,False,False,False,2022-06-30T14:27:53.393Z,2022-06-30T14:27:53.393Z,1.656614e+09,-2,-0.023,0.0,0.004147


In [150]:
df2 = df[['datetime',
          'latitude',
          'longitude', 
          'absoluteAltitude', 
          'trueHeading', 
          'groundSpeed', 
          'trueAirspeed', 
          'climbOrDescentRate', 
          'hasWeightOnWheels', 
          'turnRate to degPerSec', 
          'cyclicPositionPitch', 
          'cyclicPositionRoll', 
          'collectivePosition', 
          'antiTorquePedalPosition', 
          'throttlePosition', 
          'e1Torque', 
          'e2Torque',
          'turnDirection',
          'Altdiff',
          'groundSpeedDiff'
          
]] # only capture columns that relevant into another dataframe

# Convert timestamp into year month day and hour in seconds 
# eliminate the milliseonds part in order to group them by seconds with an average
df2['datetime'] = df2.datetime.dt.strftime('%Y-%m-%d %H:%M:%S.%f').astype('datetime64[ns]')


# Group all row by seconds with a average all the datapoint into another dataframe
df3 = df2.groupby(by=["datetime"], dropna=False).mean().reset_index()

C:\Users\hrish\AppData\Local\Temp\ipykernel_26072\4229051035.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['datetime'] = df2.datetime.dt.strftime('%Y-%m-%d %H:%M:%S.%f').astype('datetime64[ns]')


In [152]:
df3['HoverOrNot'] = 0

df3.loc[(df3['datetime'].between('2022-06-30 14:28:13' , '2022-06-30 14:33:12' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:34:10' , '2022-06-30 14:38:29' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:40:10' , '2022-06-30 14:43:40' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:46:00' , '2022-06-30 14:55:55' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:57:10' , '2022-06-30 15:00:16' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:01:20' , '2022-06-30 15:07:03' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:07:38' , '2022-06-30 15:10:38' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:12:02' , '2022-06-30 15:18:30' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:20:18' , '2022-06-30 15:23:52' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:25:30' , '2022-06-30 15:30:17' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:36:45' , '2022-06-30 15:40:24' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:42:28' , '2022-06-30 15:55:20' , inclusive='both')),['HoverOrNot']] = 1


# Double check to see hasWeightOnWheels = 1, then set Hover Or Not = 0
df3.loc[(df3['hasWeightOnWheels'] == 1), ['HoverOrNot']] = 0
df5=df3
df5.head(5)

In [284]:
from pandas.plotting import scatter_matrix
from matplotlib import cm

feature_names = [
          'absoluteAltitude', 
          'trueHeading', 
          'groundSpeed', 
          'trueAirspeed', 
          'climbOrDescentRate', 
          'turnRate to degPerSec', 
          'cyclicPositionPitch', 
          'cyclicPositionRoll', 
          'collectivePosition', 
          'antiTorquePedalPosition', 
          'throttlePosition', 
          'e1Torque', 
          'e2Torque',
          'turnDirection',
          'Altdiff',
          'groundSpeedDiff',
        'HoverOrNot']               
                
                
x = df5[feature_names]
y = df5['HoverOrNot']
len(feature_names)

#https://www.analyticsvidhya.com/blog/2019/01/introduction-time-series-classification/?
#https://www.cis.fordham.edu/wisdm/dataset.php
#https://towardsdatascience.com/time-series-classification-for-human-activity-recognition-with-lstms-using-tensorflow-2-and-keras-b816431afdff

17

In [285]:
# convert dataframe to multiple arrays

def df_to_arr_by_sequence(df_input):
    check_hover=0
    previous=0
    rotorcraft_np_arr_main=[]
    rotorcraft_np_arr_target=[]
    for i in range(len(df_input)):
        if(df_input.loc[i,"HoverOrNot"]!=check_hover):
            data = df_input.iloc[previous:i]
            data = data.drop(columns=['HoverOrNot'])
            rotorcraft_np_arr_main.append(data)
            #rotorcraft_np_arr_target.append([check_hover] * (i-previous))
            rotorcraft_np_arr_target.append(check_hover)
            #print("hover before ", check_hover)
            check_hover=1 if check_hover == 0 else 0
            #print("hover after ", check_hover)
            previous=i
    return rotorcraft_np_arr_main,rotorcraft_np_arr_target

x_main,x_target = df_to_arr_by_sequence(x)
            


            
                                
        

In [286]:
x_main = np.asarray(x_main, dtype=object)
x_target = np.asarray(x_target, dtype=np.float32)
print(x_main.size, " ", x_target.size)
pd.Series(x_main).describe()
x_target[0]

28   28


0.0

In [287]:
from keras_preprocessing.sequence import pad_sequences
#X=pad_sequences(X, maxlen=len(feature_names), padding='post', dtype='float', truncating='post')
seq_len = 100
final_seq=pad_sequences(x_main, maxlen=seq_len, padding='post', dtype='float', truncating='post')
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, len(feature_names)-1)))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [288]:
type(final_seq)
X_train = final_seq[np.r_[0:int(len(final_seq)*0.8)],:]
X_test = x_target[np.r_[0:int(len(x_target)*0.8)]]
print(len(X_train)-1)
Y_train = final_seq[np.r_[int(len(final_seq)*0.8):(len(final_seq))],:]
Y_test = x_target[np.r_[int(len(x_target)*0.8):(len(x_target))]]
print(len(final_seq))
print(len(X_train)," ",len(Y_train))
print(len(X_test)," ",len(Y_test))
#np.ndarray[0 : len(x_target)*0.8, : ]
print(Y_train.shape)

21
28
22   6
22   6
(6, 100, 16)


In [289]:

adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(X_train, X_test, epochs=200, batch_size=128, callbacks=[chk], validation_data=(Y_train,Y_test))


Epoch 1/200


C:\Users\hrish\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1/1 [==============================] - 4s 4s/step - loss: 9.4030e-04 - accuracy: 1.0000 - val_loss: 3.4938 - val_accuracy: 0.5000
Epoch 2/200
1/1 [==============================] - 0s 121ms/step - loss: 9.3596e-04 - accuracy: 1.0000 - val_loss: 3.3741 - val_accuracy: 0.5000
Epoch 3/200
1/1 [==============================] - 0s 115ms/step - loss: 8.5624e-04 - accuracy: 1.0000 - val_loss: 3.5609 - val_accuracy: 0.5000
Epoch 4/200
1/1 [==============================] - 0s 115ms/step - loss: 7.6238e-04 - accuracy: 1.0000 - val_loss: 3.6765 - val_accuracy: 0.5000
Epoch 5/200
1/1 [==============================] - 0s 114ms/step - loss: 6.7006e-04 - accuracy: 1.0000 - val_loss: 3.7207 - val_accuracy: 0.5000
Epoch 6/200
1/1 [==============================] - 0s 134ms/step - loss: 6.0505e-04 - accuracy: 1.0000 - val_loss: 3.7748 - val_accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 0s 113ms/step - loss: 5.4164e-04 - accuracy: 1.0000 - val_loss: 3.8275 - val_accuracy: 0.5000


Epoch 28/200
1/1 [==============================] - 0s 126ms/step - loss: 8.6094e-05 - accuracy: 1.0000 - val_loss: 4.5372 - val_accuracy: 0.5000
Epoch 29/200
1/1 [==============================] - 0s 149ms/step - loss: 7.9408e-05 - accuracy: 1.0000 - val_loss: 4.5690 - val_accuracy: 0.5000
Epoch 30/200
1/1 [==============================] - 0s 122ms/step - loss: 7.4086e-05 - accuracy: 1.0000 - val_loss: 4.6083 - val_accuracy: 0.5000
Epoch 31/200
1/1 [==============================] - 0s 123ms/step - loss: 7.1951e-05 - accuracy: 1.0000 - val_loss: 4.6369 - val_accuracy: 0.5000
Epoch 32/200
1/1 [==============================] - 0s 165ms/step - loss: 6.7987e-05 - accuracy: 1.0000 - val_loss: 4.6990 - val_accuracy: 0.5000
Epoch 33/200
1/1 [==============================] - 0s 180ms/step - loss: 6.4626e-05 - accuracy: 1.0000 - val_loss: 4.7279 - val_accuracy: 0.5000
Epoch 34/200
1/1 [==============================] - 0s 186ms/step - loss: 6.1593e-05 - accuracy: 1.0000 - val_loss: 4.7433 -

Epoch 55/200
1/1 [==============================] - 0s 149ms/step - loss: 2.6136e-05 - accuracy: 1.0000 - val_loss: 5.1232 - val_accuracy: 0.5000
Epoch 56/200
1/1 [==============================] - 0s 143ms/step - loss: 2.5440e-05 - accuracy: 1.0000 - val_loss: 5.1308 - val_accuracy: 0.5000
Epoch 57/200
1/1 [==============================] - 0s 131ms/step - loss: 2.4793e-05 - accuracy: 1.0000 - val_loss: 5.1365 - val_accuracy: 0.5000
Epoch 58/200
1/1 [==============================] - 0s 134ms/step - loss: 2.4194e-05 - accuracy: 1.0000 - val_loss: 5.1407 - val_accuracy: 0.5000
Epoch 59/200
1/1 [==============================] - 0s 118ms/step - loss: 2.3633e-05 - accuracy: 1.0000 - val_loss: 5.1444 - val_accuracy: 0.5000
Epoch 60/200
1/1 [==============================] - 0s 135ms/step - loss: 2.3103e-05 - accuracy: 1.0000 - val_loss: 5.1479 - val_accuracy: 0.5000
Epoch 61/200
1/1 [==============================] - 0s 134ms/step - loss: 2.2592e-05 - accuracy: 1.0000 - val_loss: 5.1512 -

Epoch 82/200
1/1 [==============================] - 0s 144ms/step - loss: 1.5233e-05 - accuracy: 1.0000 - val_loss: 5.3201 - val_accuracy: 0.5000
Epoch 83/200
1/1 [==============================] - 0s 129ms/step - loss: 1.4985e-05 - accuracy: 1.0000 - val_loss: 5.3261 - val_accuracy: 0.5000
Epoch 84/200
1/1 [==============================] - 0s 120ms/step - loss: 1.4739e-05 - accuracy: 1.0000 - val_loss: 5.3318 - val_accuracy: 0.5000
Epoch 85/200
1/1 [==============================] - 0s 125ms/step - loss: 1.4497e-05 - accuracy: 1.0000 - val_loss: 5.3373 - val_accuracy: 0.5000
Epoch 86/200
1/1 [==============================] - 0s 119ms/step - loss: 1.4262e-05 - accuracy: 1.0000 - val_loss: 5.3425 - val_accuracy: 0.5000
Epoch 87/200
1/1 [==============================] - 0s 108ms/step - loss: 1.4037e-05 - accuracy: 1.0000 - val_loss: 5.3476 - val_accuracy: 0.5000
Epoch 88/200
1/1 [==============================] - 0s 128ms/step - loss: 1.3825e-05 - accuracy: 1.0000 - val_loss: 5.3525 -

Epoch 109/200
1/1 [==============================] - 0s 129ms/step - loss: 1.0357e-05 - accuracy: 1.0000 - val_loss: 5.4353 - val_accuracy: 0.5000
Epoch 110/200
1/1 [==============================] - 0s 120ms/step - loss: 1.0220e-05 - accuracy: 1.0000 - val_loss: 5.4382 - val_accuracy: 0.5000
Epoch 111/200
1/1 [==============================] - 0s 123ms/step - loss: 1.0086e-05 - accuracy: 1.0000 - val_loss: 5.4413 - val_accuracy: 0.5000
Epoch 112/200
1/1 [==============================] - 0s 114ms/step - loss: 9.9672e-06 - accuracy: 1.0000 - val_loss: 5.4447 - val_accuracy: 0.5000
Epoch 113/200
1/1 [==============================] - 0s 127ms/step - loss: 9.8252e-06 - accuracy: 1.0000 - val_loss: 5.4486 - val_accuracy: 0.5000
Epoch 114/200
1/1 [==============================] - 0s 114ms/step - loss: 9.7010e-06 - accuracy: 1.0000 - val_loss: 5.4532 - val_accuracy: 0.5000
Epoch 115/200
1/1 [==============================] - 0s 112ms/step - loss: 9.5807e-06 - accuracy: 1.0000 - val_loss: 5

1/1 [==============================] - 0s 135ms/step - loss: 7.5973e-06 - accuracy: 1.0000 - val_loss: 5.5906 - val_accuracy: 0.5000
Epoch 136/200
1/1 [==============================] - 0s 149ms/step - loss: 7.5042e-06 - accuracy: 1.0000 - val_loss: 5.5948 - val_accuracy: 0.5000
Epoch 137/200
1/1 [==============================] - 0s 141ms/step - loss: 7.4141e-06 - accuracy: 1.0000 - val_loss: 5.5986 - val_accuracy: 0.5000
Epoch 138/200
1/1 [==============================] - 0s 135ms/step - loss: 7.3334e-06 - accuracy: 1.0000 - val_loss: 5.6020 - val_accuracy: 0.5000
Epoch 139/200
1/1 [==============================] - 0s 132ms/step - loss: 7.2564e-06 - accuracy: 1.0000 - val_loss: 5.6054 - val_accuracy: 0.5000
Epoch 140/200
1/1 [==============================] - 0s 184ms/step - loss: 7.1770e-06 - accuracy: 1.0000 - val_loss: 5.6088 - val_accuracy: 0.5000
Epoch 141/200
1/1 [==============================] - 0s 150ms/step - loss: 7.0980e-06 - accuracy: 1.0000 - val_loss: 5.6125 - val_ac

1/1 [==============================] - 0s 145ms/step - loss: 5.8303e-06 - accuracy: 1.0000 - val_loss: 5.7127 - val_accuracy: 0.5000
Epoch 163/200
1/1 [==============================] - 0s 158ms/step - loss: 5.7745e-06 - accuracy: 1.0000 - val_loss: 5.7187 - val_accuracy: 0.5000
Epoch 164/200
1/1 [==============================] - 0s 141ms/step - loss: 5.7250e-06 - accuracy: 1.0000 - val_loss: 5.7247 - val_accuracy: 0.5000
Epoch 165/200
1/1 [==============================] - 0s 149ms/step - loss: 5.6742e-06 - accuracy: 1.0000 - val_loss: 5.7304 - val_accuracy: 0.5000
Epoch 166/200
1/1 [==============================] - 0s 140ms/step - loss: 5.6163e-06 - accuracy: 1.0000 - val_loss: 5.7361 - val_accuracy: 0.5000
Epoch 167/200
1/1 [==============================] - 0s 136ms/step - loss: 5.5683e-06 - accuracy: 1.0000 - val_loss: 5.7415 - val_accuracy: 0.5000
Epoch 168/200
1/1 [==============================] - 0s 147ms/step - loss: 5.5207e-06 - accuracy: 1.0000 - val_loss: 5.7465 - val_ac

1/1 [==============================] - 0s 182ms/step - loss: 4.7142e-06 - accuracy: 1.0000 - val_loss: 5.8163 - val_accuracy: 0.5000
Epoch 189/200
1/1 [==============================] - 0s 172ms/step - loss: 4.6786e-06 - accuracy: 1.0000 - val_loss: 5.8197 - val_accuracy: 0.5000
Epoch 190/200
1/1 [==============================] - 0s 153ms/step - loss: 4.6427e-06 - accuracy: 1.0000 - val_loss: 5.8235 - val_accuracy: 0.5000
Epoch 191/200
1/1 [==============================] - 0s 166ms/step - loss: 4.6052e-06 - accuracy: 1.0000 - val_loss: 5.8279 - val_accuracy: 0.5000
Epoch 192/200
1/1 [==============================] - 0s 167ms/step - loss: 4.5637e-06 - accuracy: 1.0000 - val_loss: 5.8336 - val_accuracy: 0.5000
Epoch 193/200
1/1 [==============================] - 0s 166ms/step - loss: 4.5144e-06 - accuracy: 1.0000 - val_loss: 5.8408 - val_accuracy: 0.5000
Epoch 194/200
1/1 [==============================] - 0s 180ms/step - loss: 4.4543e-06 - accuracy: 1.0000 - val_loss: 5.8497 - val_ac

In [291]:
#model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = (model.predict(Y_train) > 0.5).astype("int32")
accuracy_score(Y_test, test_preds)

1/1 [==============================] - 0s 28ms/step


0.5